# Data Assignment 3 - `t54zheng` (20939203)


In [12]:
import pandas as pd
from pandasql import sqldf
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn import linear_model
import statsmodels.api as sm
import scipy.stats as stats
warnings.filterwarnings('ignore')

In [13]:
crsp = pd.read_excel("datasets.xlsx", sheet_name="crsp", converters={"DATE":str})
ff5x5 = pd.read_excel("datasets.xlsx", sheet_name="ff5x5")
ff3 = pd.read_excel("datasets.xlsx", sheet_name="ff3", converters={"DATEFF":str})

# Task 1 - Estimating Beta for each stock

- We wish to estimate the beta for each stock using the CAPM model for every month t between January 2000 and December 2022
- We will use an observation window of the past 36 months for each stock.

Some stocks may not have data until after Janurary 2000, so if that happens we only look at months t such that there are at least 12 observations prior to (not including) t.

In [14]:
nasdaq_100_ret = crsp[["PERMNO", "DATE", "RET"]]

In [15]:
ff3

,DATEFF,SMB,HML,MKTRF,RF,UMD
0,1980-01-31 00:00:00,0.0162,0.0175,0.0551,0.0080,0.0755
1,1980-02-29 00:00:00,-0.0185,0.0061,-0.0122,0.0089,0.0788
2,1980-03-31 00:00:00,-0.0664,-0.0101,-0.1290,0.0121,-0.0955
3,1980-04-30 00:00:00,0.0105,0.0106,0.0397,0.0126,-0.0043
4,1980-05-30 00:00:00,0.0213,0.0038,0.0526,0.0081,-0.0112
...,...,...,...,...,...,...
523,2023-08-31 00:00:00,-0.0316,-0.0106,-0.0239,0.0045,0.0377
524,2023-09-29 00:00:00,-0.0251,0.0152,-0.0524,0.0043,0.0026
525,2023-10-31 00:00:00,-0.0387,0.0019,-0.0319,0.0047,0.0173
526,2023-11-30 00:00:00,-0.0002,0.0164,0.0884,0.0044,0.0275


In [16]:
permnos = set(list(nasdaq_100_ret.PERMNO))

In [17]:
len(permnos)

99

In [8]:
# for permno in permnos:
#     permno_returns = nasdaq_100_ret.loc[nasdaq_100_ret["PERMNO"] == permno]
#     permno_returns.dropna(inplace=True)
#     permno_returns.reset_index(inplace=True) # Reset index for easy accessing later on
#     # Calculate beta using 36 month lookback starting in 2000

#     # Start date is first date where we have >= 12 observations for a beta observation
#     # Start date needs to be past 2000-01-01
#     permno_betas = {}
#     print(permno_returns.index)
#     # for t in range(12, len(permno_returns["DATE"]) - 12):
        

In [ ]:
permno_beta_dict = {}

# Filter into dicts, while removing rows with null returns
for permno in permnos:
    permno_returns = nasdaq_100_ret.loc[nasdaq_100_ret["PERMNO"] == permno]
    permno_returns.dropna(inplace=True)
    permno_returns.reset_index(inplace=True) # Reset index for easy accessing later on

    # Calculate beta using 36 month lookback starting in 2000

    # Start date is first date where we have >= 12 observations for a beta observation
    # Start date needs to be past 2000-01-01
    permno_betas = {}
    for t in range(12, len(permno_returns["DATE"]) - 12):
        # Don't predict if date is not on or after 2000-01-01
        if permno_returns.iloc[t].DATE < "2000-01-01":
            continue

        date = permno_returns.iloc[t].DATE
        est_window_start = t - 35 if t > 35 else 0

        est_window_end = t # In indexing actually only goes up to t-1

        
        est_window_returns = permno_returns.iloc[est_window_start:est_window_end].RET
        est_window_returns_dates = permno_returns.iloc[est_window_start:est_window_end].DATE

        date_mask = ff3["DATEFF"].isin(est_window_returns_dates)
        rf = ff3[date_mask]["RF"]
        mktrf = ff3[date_mask][["MKTRF"]]

        CAPMmodel = linear_model.LinearRegression().fit(mktrf, est_window_returns.values - rf.values)

        permno_betas[date] = CAPMmodel.coef_[0]
    permno_beta_dict[permno] = permno_betas


In [26]:
for permno in permno_beta_dict:
    permno_beta_dict[permno] = pd.DataFrame.from_dict(permno_beta_dict[permno], orient="index", columns = ["DATE", "beta"])

SyntaxError: positional argument follows keyword argument (1000966794.py, line 2)

In [27]:
permno_beta_dict[87055]

,0
2000-01-31 00:00:00,1.243951
2000-02-29 00:00:00,1.155894
2000-03-31 00:00:00,1.238501
2000-04-28 00:00:00,1.234853
2000-05-31 00:00:00,1.122355
...,...
2021-08-31 00:00:00,0.516960
2021-09-30 00:00:00,0.520213
2021-10-29 00:00:00,0.520876
2021-11-30 00:00:00,0.542254
